## Network Validation Checks

#### Import Libraries 

In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

#### Load Network data from Shapefiles

In [2]:
line_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [3]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

In [4]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    #Convert to Degrees and Return
    return angle_radians * 180 / np.pi

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [5]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp_load)


# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [6]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [7]:
def link_matcher_atts(*atts, graph, edge_id='ID'):
    node_feed = []

    for node in node_shp_load['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]                
                node_feed.append([node, 
                                  *[d[edge_id] for d in data],
                                  *[d[a] for a in atts for d in data],
#                                   *[o_data[a] - i_data[a] for a in atts]
                                 ])

    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node',
            *[ d+'_'+edge_id for d in dirs],
            *[ d+'_'+a for a in atts for d in dirs],
#             *['diff_'+a for a in atts]
           ]

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_filter_atts*: 
Apply Attribute to filter by difference value 

In [8]:
def attribute_change(network, attributes):
    for att in attributes:
        in_fld = 'in_{}'.format(att)
        print(network[in_fld][0])
        out_fld = 'out_{}'.format(att)
        fld_diff = 'diff_{}'.format(att)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(network[in_fld][0], (int, float)):
            network[fld_diff] = network[out_fld] - network[in_fld]
        elif isinstance(network[in_fld][0], str):
            network[fld_diff] = 'SAME' if network[out_fld].str == network[in_fld].str else 'CHANGES'

    return network#.groupby([in_fld, out_fld]).count().reset_index()

In [10]:
def attribute_filtering(network, att_filters, get_node_max):
    for filter_name, filter_val in att_filters.items():
        in_fld = 'in_{}'.format(filter_name)
        out_fld = 'out_{}'.format(filter_name)
        fld_diff = 'diff_{}'.format(filter_name)
#         abs_fld_diff = 'abs_diff_{}'.format(filter_name)
    
        if isinstance(filter_val, (int, float)):
            network = network[abs(network[fld_diff]) >= filter_val]
        elif isinstance(filter_val, str):
            network = network[(network[in_fld] == filter_val) | (network[out_fld] == filter_val)]

    return network#.groupby([in_fld, out_fld]).count().reset_index()

In [11]:
# In/Out Long Table
def export_geo_compare(network, att_name, att_name_add=None):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'geometry']
#     if att_name_add: geo_cols.append(att_name_add)
    network = pd.merge(line_shp_load[geo_cols], network, on='ID')
    
    shp_export = os.path.join('..', 'data', 'Network_attribute_change_{}.shp'.format('_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *link_compare*: 
Call functions to calculate Link-to-Link attribute comparison

In [25]:
def link_compare(attribute_names, attribute_filters, get_node_max):
    network = link_matcher_atts(*attribute_names, graph=g)
    network = attribute_change(network, attribute_names)
    network = attribute_filtering(network,
                                attribute_filters,
                                get_node_max
                                )
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format('_'.join([n for n in attribute_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total {} network links with change in {}: {:,}'
    print(print_txt.format('SUMMARIZED' if get_node_max else '\b',
                           ', '.join([n for n in attribute_names]),
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if get_node_max:
        network, shp_export = export_geo_compare(network, attribute_filters)
        print('Shapefile exported to {}'.format(shp_export))

    return network

## VALIDATION TESTS

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and FUNCTIONAL CLASS

In [27]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', '_LANES']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    '_LANES':2
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

45.0
2.0
Total SUMMARIZED network links with change in SPD_LMT, _LANES: 238
Table exported to ..\data\Network_attribute_change_table_SPD_LMT__LANES.csv
Shapefile exported to ..\data\Network_attribute_change_SPD_LMT__LANES.shp


,ID,geometry,node,in_ID,in_SPD_LMT,out_SPD_LMT,in__LANES,out__LANES,diff_SPD_LMT,diff__LANES
0,153,"LINESTRING (2209220.826 268047.894, 2209150.14...",2079,55,45.0,55.0,1.0,3.0,10.0,2.0
1,667,"LINESTRING (2209182.192 268014.816, 2209077.66...",2613,586,45.0,55.0,1.0,3.0,10.0,2.0
2,2136,"LINESTRING (2169602.094 279040.445, 2169562.60...",2746,723,30.0,45.0,1.0,3.0,15.0,2.0
3,15965,"LINESTRING (2132079.841 230583.789, 2132186.81...",15215,17654,65.0,45.0,3.0,1.0,-20.0,-2.0
4,1239,"LINESTRING (2169495.944 279250.838, 2169244.58...",2746,723,30.0,45.0,1.0,3.0,15.0,2.0


### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Speed Limit

In [21]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

45.0
CENTROID CONNECTOR
Total  network links with change in SPD_LMT, RTE_NME: 7,791
Table exported to ..\data\Network_attribute_change_table_SPD_LMT_RTE_NME.csv


,node,in_ID,out_ID,in_SPD_LMT,out_SPD_LMT,in_RTE_NME,out_RTE_NME,diff_SPD_LMT,diff_RTE_NME
14,11161,38587,10186,45.0,55.0,CENTROID CONNECTOR,GEORGETOWN RD NW,10.0,CHANGES
15,11161,10186,38587,55.0,45.0,GEORGETOWN RD NW,CENTROID CONNECTOR,-10.0,CHANGES
18,2158,284,110,45.0,65.0,None,US-27,20.0,CHANGES
22,2674,635,782,65.0,45.0,US-27,None,-20.0,CHANGES
158,2523,2308,487,45.0,55.0,TN-58,TN-58,10.0,CHANGES


### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Route Name

In [23]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    'SPD_LMT':10,
    'RTE_NME':'CENTROID CONNECTOR',
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

45.0
CENTROID CONNECTOR
Total  network links with change in SPD_LMT, RTE_NME: 5,522
Table exported to ..\data\Network_attribute_change_table_SPD_LMT_RTE_NME.csv


,node,in_ID,out_ID,in_SPD_LMT,out_SPD_LMT,in_RTE_NME,out_RTE_NME,diff_SPD_LMT,diff_RTE_NME
14,11161,38587,10186,45.0,55.0,CENTROID CONNECTOR,GEORGETOWN RD NW,10.0,CHANGES
15,11161,10186,38587,55.0,45.0,GEORGETOWN RD NW,CENTROID CONNECTOR,-10.0,CHANGES
486,7410,5814,37902,35.0,45.0,DELASHMITT RD,CENTROID CONNECTOR,10.0,CHANGES
487,7410,37902,5779,45.0,35.0,CENTROID CONNECTOR,DELASHMITT RD,-10.0,CHANGES
489,7410,37902,5814,45.0,35.0,CENTROID CONNECTOR,DELASHMITT RD,-10.0,CHANGES


### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTIONAL CLASS
Filter by Number of Lanes

In [24]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_names = ['_LANES', 'FUNCCLASS']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filters = {
    '_LANES':2,
}
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attribute_names, attribute_filters, get_node_max)
network.head()

2.0
98.0
Total  network links with change in _LANES, FUNCCLASS: 776
Table exported to ..\data\Network_attribute_change_table__LANES_FUNCCLASS.csv


,node,in_ID,out_ID,in__LANES,out__LANES,in_FUNCCLASS,out_FUNCCLASS,diff__LANES,diff_FUNCCLASS
59,3175,13886,2063,1.0,3.0,19.0,14.0,2.0,-5.0
65,3175,11550,2063,1.0,3.0,19.0,14.0,2.0,-5.0
91,3734,1689,11674,3.0,1.0,14.0,19.0,-2.0,5.0
98,3734,1690,11674,3.0,1.0,14.0,19.0,-2.0,5.0
102,3734,11674,1690,1.0,3.0,19.0,14.0,2.0,-5.0


### Put attributes on the network

In [ ]:
node_shp['X'] = node_shp.geometry.x
node_shp['Y'] = node_shp.geometry.y
nx.set_node_attributes(g, pd.Series(node_shp.X, index=node_shp.ID).to_dict(), 'X')
nx.set_node_attributes(g, pd.Series(node_shp.Y, index=node_shp.ID).to_dict(), 'Y')

### Visualizations (Slow- Due to Network Size)

In [ ]:
# plt.subplot(121)

#nx.draw(g, with_labels=True, font_weight='bold')
#plt.subplot(122)

#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(g, nx.spring_layout(g), node_size=10, node_color='blue')
plt.show()
